Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import torch
import transformers
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
from tqdm import notebook

ERROR! Session/line number was not unique in database. History logging moved to new session 2


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Основная идея нижеследующих блоков в том, чтобы обучить модель без нейронок, посмотреть, сможем ли выжать приемлемый результат без них (хорошо бы, если так), а потом повторить обучение на PyTorch+BERT

In [5]:
train, test = train_test_split(df, random_state=12345)

# 2. Обучение классическим ML

В этом блоке проведем обучение и тест без нейронных сетей

In [6]:
stemmer = SnowballStemmer('english') 

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
def lemmatize(text):
    #lemm_list = stemmer.lemmatize(text)
    lemm_list = [stemmer.stem(word) for word in text]
    lemm_text = "".join(lemm_list)
        
    return lemm_text

def clear_text(text):
    reed = re.sub(r'[^a-zA-Z ]', ' ', text)
    splitted = reed.split()
    joined = " ".join(splitted)
    return joined

Делаем копии выборок, то же сделаем для блока с нейронками, для лучшей сравнимости результатов

In [9]:
train_tweets, test_tweets = train.copy(), test.copy()

Применяем лемматизацию и очистку

In [10]:
train_tweets['lemm_text'] = train['text'].apply(lemmatize)
train_tweets['lemm_text'] = train_tweets['lemm_text'].apply(clear_text)
test_tweets['lemm_text'] = test['text'].apply(lemmatize)
test_tweets['lemm_text'] = test_tweets['lemm_text'].apply(clear_text)

Загружаем список стоп-слов из библиотеки NTLK

In [11]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
corpus_train, corpus_test = train_tweets['lemm_text'].values.astype('U'), \
                            test_tweets['lemm_text'].values.astype('U')

Производим векторизацию (при этом fit делаем только на обучающей выборке)

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

<h4> Модель логистической регрессии </h4>

In [14]:
model = LogisticRegression(max_iter=300, solver='liblinear', class_weight='balanced')
score1 = cross_val_score(model, tf_idf_train, train.toxic, scoring='f1')

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [15]:
print(np.mean(score1))

0.7472438636262243


In [16]:
model.fit(tf_idf_train, train.toxic)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=300, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
pred_lr = model.predict(tf_idf_test)

In [18]:
 print("F1:", f1_score(test.toxic, pred_lr).round(4), '\nPrecision:',
       precision_score(test.toxic, pred_lr).round(4), '\nRecall', recall_score(test.toxic, pred_lr).round(4))

F1: 0.7588 
Precision: 0.6829 
Recall 0.8537


In [19]:
print(confusion_matrix(test.toxic, pred_lr))

[[34186  1620]
 [  598  3489]]


<h4> Градиентный бустинг на деревьях </h4>

В целях экономии времени, я решил остановиться на одной модели бустинга

In [20]:
model_lgmb = lgb.LGBMClassifier(n_estimators=500)

In [2]:
score2 = cross_val_score(model_lgmb, tf_idf_train, train.toxic, scoring='f1')

In [ ]:
print(np.mean(score2).round(4))

In [ ]:
df.toxic.mean()

# 3. Neural Network

Я очень хотел применить нейронки, в итоге не смог получить какой-то выдающийся результат, в лучшем случае F1=0.72. Была испробована модель bert_uncased и вторая, distilbert, используемая ниже. В документации сказано, она хорошо подходит для семантического анализа и более легковесна чем собсвенно BERT. Эмбединги генерировались около 3 часов, поэтому я сохранил результат в csv-файл и в представленной версии ноутбука признаки и ответы берутся оттуда. Сам процесс обучения оставляю в закомментированном виде.

In [47]:
# df_tweets = train_tweets.sample(30000).reset_index(drop=True)

In [48]:
# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# pt_model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, model_max_length=512, truncation=True)

In [49]:
# tensors = tokenizer(df_tweets.lemm_text.to_list(), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [50]:
# batch_size = 10
# embeddings = []
# for i in notebook.tqdm(range(df_tweets.shape[0] // batch_size)):
    
#         tmp_batch = tensors.data['input_ids'][batch_size*i:batch_size*(i+1)]
#         attention_mask_batch = tensors.data['attention_mask'][batch_size*i:batch_size*(i+1)]
        
#         with torch.no_grad():
#             batch_embeddings = pt_model(tmp_batch, output_hidden_states=True, output_attentions=True,
#                                        attention_mask=attention_mask_batch)

#         embeddings.append(batch_embeddings[1][-1][:,0,:].numpy())

## Важный дисклеймер

Эмбединги сверху считаются на моем ПК около трех часов (с учетом того, что я не являюсь обладателем даже consumer-grade видеокарты Nvidia, не говоря уже о чем-то более серьезном). Выборка неполная (30000 строк), но качество вряд ли станет лучше на полной (и у меня столько оперативной памяти нет).

Поэтому сгрузил готовые эмбеддинги в виде файлов csv. К сожалению, облака не предоставляют прямые ссылки на файлы, а гитхаб такие большие и вовсе принимать отказывается. Пришлось прибегнуть к помощи Google Colab, там готовая среда и точно все запустится.

Правда, проблему с подгрузкой файла не и там не удалось как-то красиво решить.

Поэтому, если есть возможность, посмотрите вторую часть по ссылке ниже. Для просмотра никаких доп. манипуляций не требуется, однако если вы сочтете возможным, там есть мини-инструкция по подгрузке файлов с эмбеддингами, после чего код можно выполнять.

Очень много времени потратил на работу с этой моделью, не хотелось выбрасывать, даже несмотря на не самый лучший результат.

Спасибо за понимание. Потом можно сразу пролистать к выводам.

https://colab.research.google.com/drive/1XTw1X7BS-_p9h03uH0ffCf_9BTzr7ouS?usp=sharing

In [51]:
# features = np.concatenate(embeddings)
# target = df_tweets['toxic']

In [52]:
# features = np.loadtxt(r'D:\features_new_new.csv')

In [53]:
# target = pd.read_csv(r'D:\target_new.csv').values

In [54]:
# model2 = LogisticRegression(max_iter=500, random_state=12345, solver='liblinear', C=0.7)

In [55]:
# print(cross_val_score(model2, features, target, scoring='f1').mean())

Сплит, чтобы посмотреть характер ошибок модели

In [56]:
# X_train, X_test, y_train, y_test = train_test_split(features, target, stratify=target, test_size=0.1, random_state=12345)

"Ручной" фит + предикт

In [57]:
# model2.fit(X_train, y_train)
# pred_nn = model2.predict(X_test)

Результаты:

In [58]:
# print("F1:", f1_score(y_test, pred_nn).round(4), '\nPrecision:',
#       precision_score(y_test, pred_nn).round(4), '\nRecall', recall_score(y_test, pred_nn).round(4))

Матрица ошибок:

In [59]:
# print(confusion_matrix(y_test, pred_nn))

Целевой класс в трети случаев не опознан - плохая модель, в прод я бы такое постыдился выпускать

Получим вероятности

In [60]:
# probas = model2.predict_proba(X_test)

Небольшой перебор порогов классификации

In [61]:
# for i in np.arange(0.2, 0.8, 0.1):
#     preds_by_hand = [0 if pred_pos < i else 1 for pred_neg, pred_pos in probas]
#     print(i, 'is', f1_score(y_test, preds_by_hand))

In [62]:
# features = pd.read_csv(r'D:\features_new.csv')

Таки дожали 0.75, но в целом провал

In [63]:
# test = pd.DataFrame(df_tweets['text'])

In [64]:
# test_train, test_test = train_test_split(test, test_size=0.1, random_state=12345)

In [65]:
# test_test['real'] = y_test

In [66]:
# test_test['pred'] = pred_nn

# 4. Выводы

В данном проекте мы решали задачу семантической классификации текстов. В ходе работы обучили модели логистической регрессии и градиентного бустинга на деревьях, применив TD-IDF и очистку, используя регулярные выражения. Необходимый уровень целевой метрики F1 был достигнут.

Очень много времени было потрачено на нейросеть BERT (точнее, я использовал вариант Distibert, согласно документации эта модель хорошо подходит именно для задачи классификации. хотя обычный BERT тоже пробовал). К сожалению, в итоге данная модель все равно оказалась сравнима или даже хуже простой логистической регрессии (или бустинга на деревьях), а с учетом времени обучения и затрат вычислительных ресурсов ее попросту нецелесобразно пускать в продакшн. В качестве возможной причины тауой низкой результативности я бы обозначил шум в данных, в частности, я выборочно просмотрел тексты, попадаются и куски иноязычного текста попадаются (не английский), и разного рода сленг. Кроме того, отдельно посмотрел примеры, где модель не смогла правильно выделить целевой класс (ложноотрицательные ответы), если бы я размечал данные, не все бы пометил как токсичные. Т.е. задача неочевидная даже с точки зрения человека, что уж говорить о машине.

Преподаватель также высказывался, что некоторые датасеты с легкостью берет обычная регрессия, а другие даже  лучшим моделям не под силу. У меня впечатление, что мы имеем второй случай. Думаю, модель BERT может выдавать гораздо лучший результат, а если запускать ее на подходящем оборудовании, то и с производительностью не будет больших проблем.